## Image classification with deep learning methods.

-- Description --

When you train the network, it is recommended to use the GPU resources of your computer.
This will help you to learn the "know how" of setting up a working Python environment on your computer.
In the case of unavailable Nvidia hardware or problems with your Python environment you can use Google Colab.
Please go to the menu, Runtime - Change runtime type, and select **GPU** as the hardware accelerator.
Although you used your computer successfuly it is highly recommended to give a try to Google Colab environment.


In [ ]:
# Import libraries
# These libraries should be sufficient for this Practice.
# However, if any other library is needed, please install it by yourself.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torch.utils.data as data
import numpy as np
import time
import os
import random
import matplotlib.pyplot as plt
from matplotlib import colors
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import tensorflow as tf

!pip install medmnist
import medmnist
from medmnist import *

In [ ]:
# Parameters
NUM_EPOCHS = 20
BATCH_SIZE = 32
lr = 0.0001
DOWNLOAD_OK = True
data_flag = 'bloodmnist'
im_size = 224
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
num_classes = len(info['label'])
N_IMAGES = 1000
data_labels = info['label']

# Tupla que contiene los valores asociados a los parámetros mostrados anteriormente,
# con el fin de mejorar la organización de dichos parámetros.
parameters = {"num_epochs": NUM_EPOCHS, "batch_size": BATCH_SIZE, "lr": lr, "download_ok": DOWNLOAD_OK,
              "data_flag": data_flag, "im_size": im_size,"info_task": task, "n_channels": n_channels,
              "num_classes": num_classes,"n_images":N_IMAGES, "data_labels": data_labels}

# Preprocesado de datos mediante la definición de la transformación de datos
def preprocessing_data(parameters):
  data_transform = transforms.Compose([
    transforms.Resize((parameters["im_size"],parameters["im_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
  ])

  full_train_dataset = BloodMNIST(split="train", transform=data_transform, download=True)
  full_valid_dataset = BloodMNIST(split="val", transform=data_transform, download=True)
  full_test_dataset = BloodMNIST(split="test", transform=data_transform, download=True)

  idx_train = np.random.choice(len(full_train_dataset),size=1000,replace=False)
  train_dataset = [full_train_dataset[i] for i in idx_train]

  idx_valid = np.random.choice(len(full_valid_dataset),size=300,replace=False)
  valid_dataset = [full_valid_dataset[i] for i in idx_valid]

  idx_test = np.random.choice(len(full_test_dataset),size=600,replace=False)
  test_dataset = [full_test_dataset[i] for i in idx_test]

  train_loader = data.DataLoader(dataset=train_dataset, batch_size=parameters["batch_size"], shuffle=True)
  valid_loader = data.DataLoader(dataset=valid_dataset, batch_size=parameters["batch_size"], shuffle=False)
  test_loader = data.DataLoader(dataset=test_dataset, batch_size=parameters["batch_size"], shuffle=True)

  return train_loader, valid_loader, test_loader

train_loader,valid_loader,test_loader = preprocessing_data(parameters)

Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz


In [ ]:
## Your code

# Función que permite visualizar los aspectos fundamentales sobre cada
# dataset que se haya descargado/cargado previamente.
def dataset_visualizer(dataset,length_montage):
  for i in range(0,length_montage*length_montage):
    img = dataset[i][0]
    label = str(dataset[i][1]).replace('[','').replace(']','')
    figure = plt.figure(figsize=(2,2))
    plt.imshow(img.permute(1,2,0))
    plt.title(data_labels[label])
    plt.axis("off")
  plt.show()

# Visualizador de las imágenes a través de un pipeline DataLoader
def dataloader_visualizer(data_loader,num_batches):
  for batch_idx, (features, labels) in enumerate(data_loader):
      if batch_idx >= num_batches:
        break
      for i in range(len(features)):
        img = features[i].squeeze()
        label = str(labels[i]).replace('tensor([','').replace('])','')
        plt.figure(figsize=(2,2))
        plt.title(label)
        plt.imshow(img.permute(1,2,0))
        plt.axis('off')
        plt.show()

def net_model_visualizer(net_model):
  print(net_model)
  print('Total Parameters:',
       sum([torch.numel(p) for p in net_model.parameters()])
  )
  print('Trainable Parameters:',
       sum([torch.numel(p) for p in net_model.parameters() if p.requires_grad])
  )

#dataloader_visualizer(train_loader,1)
#dataloader_visualizer(valid_loader,1)
#dataloader_visualizer(test_loader,1)

In [ ]:
# Define a simple CNN model

def convolution(in_channels,out_channels):
  layer = nn.Sequential(
      nn.Conv2d(in_channels,out_channels,3),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )

  return layer

class Net(nn.Module):
    def __init__(self, parameters):
        super(Net, self).__init__()
        #Define the desired deep learning model
        #Your code

        self.layer1 = nn.Sequential(
            nn.Conv2d(parameters["n_channels"],32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(128,256,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.fc1 = nn.Linear(256,128)
        self.drop1 = nn.Dropout(0.33)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128,parameters["num_classes"])

        #End your code

    def forward(self, x):
        #Your code
        #print(1, x.shape)
        x = self.layer1(x)
        #print(2, x.shape)
        x = self.layer2(x)
        #print(3, x.shape)
        x = self.layer3(x)
        #print(4, x.shape)
        x = self.layer4(x)

        #print(5, x.shape)
        x = F.adaptive_avg_pool2d(x,1).reshape(x.size(0),-1)

        #print(6, x.shape)
        x = self.fc1(x)
        x = self.drop1(x)
        x = self.relu1(x)
        #print(7, x.shape)
        x = self.fc2(x)
        #print(8, x.shape)

        return x

#def double_convultion(in_channels,out_channels):

#  conv_op = nn.Sequential(
#      nn.Conv2d(in_channels,out_channels,1),
#      nn.ReLU(True),
#      nn.Conv2d(out_channels,out_channels,1),
#      nn.ReLU(True)
#  )

#  return conv_op

#class UNetCNN(nn.Module):
#  def __init__(self,parameters):
#    super(UNetCNN,self).__init__()

#    self.max_pool2d = nn.MaxPool2d(2,2)

#    self.down_conv1 = double_convultion(parameters["n_channels"],64)
#    self.down_conv2 = double_convultion(64,128)
#    self.down_conv3 = double_convultion(128,256)
#    self.down_conv4 = double_convultion(256,512)
#    self.down_conv5 = double_convultion(512,1024)

#    self.up_trans1 = nn.ConvTranspose2d(1024,512,2,2)
#    self.up_conv1 = double_convultion(1024,512)
#    self.up_trans2 = nn.ConvTranspose2d(512,256,2,2)
#    self.up_conv2 = double_convultion(512,256)
#    self.up_trans3 = nn.ConvTranspose2d(256,128,2,2)
#    self.up_conv3 = double_convultion(256,128)
#    self.up_trans4 = nn.ConvTranspose2d(128,64,2,2)
#    self.up_conv4 = double_convultion(128,64)

#    self.final_layer = nn.Conv2d(64,parameters["num_classes"],1)

#  def forward(self,x):
#    print(x.shape)
#    down1 = self.down_conv1(x)
#    print(down1.shape)
#    down2 = self.max_pool2d(down1)
#    print(down2.shape)
#    down3 = self.down_conv2(down2)
#    print(down3.shape)
#    down4 = self.max_pool2d(down3)
#    print(down4.shape)
#    down5 = self.down_conv3(down4)
#    print(down5.shape)
#    down6 = self.max_pool2d(down5)
#    print(down6.shape)
#    down7 = self.down_conv4(down6)
#    print(down7.shape)
#    down8 = self.max_pool2d(down7)
#    print(down8.shape)
#    down9 = self.down_conv5(down8)
#    print(down9.shape)

#    up1 = self.up_trans1(down9)
#    print(up1.shape)
#    x = self.up_conv1(torch.cat([down7,up1],1))

#    up2 = self.up_trans2(x)
#    print(up2.shape)
#    x = self.up_conv2(torch.cat([down5,up2],1))

#    up3 = self.up_trans3(x)
#    print(up3.shape)
#    x = self.up_conv3(torch.cat([down3,up3],1))

#    up4 = self.up_trans4(x)
#    print(up4.shape)
#    x = self.up_conv4(torch.cat([down1,up4],1))

#    final_layer = self.final_layer(x)
#    return final_layer

#class ResNet18(nn.Module):
#  def __init__(self,parameters):
#    super(ResNet18, self).__init__()

#    self.conv1 = nn.Conv2d(parameters["n_channels"],64,7,2,3)
#    self.bn1 = nn.BatchNorm2d(64)
#    self.maxpool = nn.MaxPool2d(3,2,1)

#    self.conv2_block1 = nn.Sequential(
#        nn.Conv2d(64,64,3,1,1),
#        nn.BatchNorm2d(64),
#        nn.ReLU(True),
#        nn.Conv2d(64,64,3,1,1),
#        nn.BatchNorm2d(64)
#    )

#    self.conv2_block2 = nn.Sequential(
#        nn.Conv2d(64,64,3,1,1),
#        nn.BatchNorm2d(64),
#        nn.ReLU(True),
#        nn.Conv2d(64,64,3,1,1),
#        nn.BatchNorm2d(64)
#    )

#    self.conv3_block1 = nn.Sequential(
#        nn.Conv2d(64,128,3,2,1),
#        nn.BatchNorm2d(128),
#        nn.ReLU(True),
#        nn.Conv2d(128,128,3,1,1),
#        nn.BatchNorm2d(128)
#    )

#    self.conv3_block2 = nn.Sequential(
#        nn.Conv2d(128,128,3,1,1),
#        nn.BatchNorm2d(128),
#        nn.ReLU(True),
#        nn.Conv2d(128,128,3,1,1),
#        nn.BatchNorm2d(128)
#    )

#    self.conv4_block1 = nn.Sequential(
#        nn.Conv2d(128,256,3,2,1),
#        nn.BatchNorm2d(256),
#        nn.ReLU(True),
#        nn.Conv2d(256,256,3,1,1),
#        nn.BatchNorm2d(256)
#    )

#    self.conv4_block2 = nn.Sequential(
#        nn.Conv2d(256,256,3,1,1),
#        nn.BatchNorm2d(256),
#        nn.ReLU(True),
#        nn.Conv2d(256,256,3,1,1),
#        nn.BatchNorm2d(256)
#    )

#    self.conv5_block1 = nn.Sequential(
#        nn.Conv2d(256,512,3,2,1),
#        nn.BatchNorm2d(512),
#        nn.ReLU(True),
#        nn.Conv2d(512,512,3,1,1),
#        nn.BatchNorm2d(512)
#    )

#    self.conv5_block2 = nn.Sequential(
#        nn.Conv2d(512,512,3,1,1),
#        nn.BatchNorm2d(512),
#        nn.ReLU(True),
#        nn.Conv2d(512,512,3,1,1),
#        nn.BatchNorm2d(512)
#    )

#    self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
#    self.fc = nn.Linear(512,1000)

#  def forward(self,x):
#    print(x.shape)
#    x = self.conv1(x)
#    x = self.bn1(x)
#    x = self.maxpool(x)

#    print(x.shape)
#    x = self.conv2_block1(x)
#    x = self.conv2_block2(x)
#    x = self.conv3_block1(x)
#    x = self.conv3_block2(x)
#    x = self.conv4_block1(x)
#    x = self.conv4_block2(x)
#    x = self.conv5_block1(x)
#    x = self.conv5_block2(x)

#    x = self.avgpool(x)
#    x = self.fc(x)

#    return x

        #End your code

In [ ]:
# Train the model
def train_epoch(model,train_loader,optimizer,criterion,task,epoch,parameters):
    train_acc = 0
    train_loss = 0

    for batch_idx, (X_batch, y_batch) in tqdm(enumerate(train_loader),total=len(train_loader)):
        outputs = model(X_batch)
        y_batch = y_batch.squeeze().long()
        y_batch = F.one_hot(y_batch, num_classes=parameters["num_classes"])

        #print(outputs.shape)
        #print(y_batch.shape)

        loss = criterion(outputs,y_batch.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += torch.sum(outputs == y_batch.data).float().item()
        train_loss += loss.item()

    train_acc /= len(train_loader.dataset)
    train_loss /= len(train_loader)

    return{
        "epoch": epoch,
        "train_acc": train_acc * 100,
        "train_loss": train_loss
    }

def validate_epoch(model,valid_loader,criterion,task,epoch,parameters):
    valid_acc = 0
    valid_loss = 0

    for batch_idx, (X_batch, y_batch) in tqdm(enumerate(valid_loader),total=len(valid_loader)):
        with torch.no_grad():
          outputs = model(X_batch)
          y_batch = y_batch.squeeze().long()
          y_batch = F.one_hot(y_batch, num_classes=parameters["num_classes"])

          loss = criterion(outputs,y_batch.float())

          _, preds = torch.max(outputs,1)

          valid_acc += torch.sum(outputs == y_batch.data).float().item()
          valid_loss += loss.item()

    valid_loss /= len(valid_loader)
    valid_acc /= len(valid_loader.dataset)

    return {
        "epoch" : epoch,
        "valid_acc": 100 * valid_acc,
        "batch_loss": valid_loss
    }


def model_training(train_loader,valid_loader,parameters):
  model = Net(parameters)
  #model = ResNet18(parameters)
  #model = UNetCNN(parameters)

  optimizer = torch.optim.Adam(model.parameters(),lr=parameters["lr"])
  criterion = nn.CrossEntropyLoss()

  #net_model_visualizer(model)

  for epoch in range(parameters["num_epochs"]):
    model.train()
    train_history = train_epoch(model,train_loader,optimizer,criterion,task,epoch,parameters)
    print(train_history)
    model.eval()
    valid_history = validate_epoch(model,valid_loader,criterion,task,epoch,parameters)
    print(valid_history)

model_training(train_loader,valid_loader,parameters)

100%|██████████| 63/63 [02:02<00:00,  1.94s/it]


{'epoch': 0, 'train_acc': 0.0, 'train_loss': 2.048327254870581}


100%|██████████| 19/19 [00:15<00:00,  1.26it/s]


{'epoch': 0, 'valid_acc': 0.0, 'batch_loss': 1.9437567623038041}


100%|██████████| 63/63 [02:05<00:00,  1.99s/it]


...

{'epoch': 18, 'valid_acc': 0.0, 'batch_loss': 0.6953602511631815}


100%|██████████| 63/63 [01:59<00:00,  1.89s/it]


{'epoch': 19, 'train_acc': 0.0, 'train_loss': 0.8612980800015586}


100%|██████████| 19/19 [00:15<00:00,  1.25it/s]

{'epoch': 19, 'valid_acc': 0.0, 'batch_loss': 0.693356088901821}


#Evaluation

Finally, implement the evaluation of the object clasification task. You can implement any metric you want, though the most common are accuracy and AUC (one class against all for the multiclass task). You can use torch.no_grad() for speeding up predictions when no gradients are needed.

How do you compare with the MedMNIST benchmarks?